In [15]:
# d for coin futures, f for usd s futures
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import requests as re
import pandas as pd
import numpy as np

import multiprocess as mp

COIN_API = 'https://dapi.binance.com'
USD_API = 'https://fapi.binance.com'
SPOT_API = 'https://api.binance.com'
g_api_key='eJBYUpKfnUCA5wXDfJTKGPNaOoyLtnXIb9D6GKFbnABLCndVRRYJrdyI9rMi6D54'
g_secret_key='970sbCiGtH1W4rPW56I4MBbD9brKZegyveedljt3U3D5hry7zMbxa4sxacA80kUv'
headers = {"Accept": "application/json"}

ModuleNotFoundError: No module named 'multiprocess'

In [11]:
epoch = datetime.utcfromtimestamp(0)
def dt_from_ms(ms):
    return datetime.utcfromtimestamp(int(ms / 1000.0))

def dt_to_ms(dt):
    delta = dt - epoch
    return int(delta.total_seconds() * 1000)

def get_pairs(isCoin, isSpot=False):
    if isSpot:
        suff = '/api/v3/exchangeInfo'
        url = SPOT_API
        result = re.get(url+suff, headers=headers)
        symbolObjs = result.json()['symbols']
        symbolObjs = list(filter(lambda x: x['symbol'][-4:] == 'USDT', symbolObjs))
        symbolObjs = [x['symbol'] for x in symbolObjs]
        return {'spot': symbolObjs}
    else:
        suff = '/dapi/v1/exchangeInfo' if isCoin else '/fapi/v1/exchangeInfo'
        url = COIN_API if isCoin else USD_API
        result = re.get(url+suff, headers=headers)
        symbolObjs = result.json()['symbols']
        symbols = [x['symbol'] for x in symbolObjs]
        perpetuals = list(filter(lambda x: 'PERP' in x, symbols))
        futures = list(filter(lambda x: 'PERP' not in x, symbols))
        return {'perpetuals': perpetuals, 'futures': futures}

def get_funding_rate(isCoin, pair, start_date, end_date, limit=1000):
    start_year, start_month, start_day = int(start_date[0:4]), int(start_date[4:6]), int(start_date[6:8])
    end_year, end_month, end_day = int(end_date[0:4]), int(end_date[4:6]), int(end_date[6:8])
    start_date = dt_to_ms(datetime(start_year, start_month, start_day))
    end_date = dt_to_ms(datetime(end_year, end_month, end_day))
    suff = '/dapi/v1/fundingRate' if isCoin else '/fapi/v1/fundingRate'
    suff += '?symbol='+pair+'&startTime='+str(start_date)+'&endTime='+str(end_date)+'&limit='+str(limit)
    url = COIN_API if isCoin else USD_API
    result = re.get(url+suff, headers=headers)
    data = result.json()
    funding_rate, funding_time = [], []
    for funding in data:
        funding_rate.append(float(funding['fundingRate'])*100)
        funding_time.append(dt_from_ms(funding['fundingTime']))
    df = pd.DataFrame({'Time':funding_time,'Rate':funding_rate})
    df['Time'] = pd.to_datetime(df['Time'])
    df.set_index('Time',inplace=True)
    return df

def get_kline_data(isCoin, pair, start_date, end_date, limit=1500, interval='1h', isSpot=False):
    start_year, start_month, start_day = int(start_date[0:4]), int(start_date[4:6]), int(start_date[6:8])
    end_year, end_month, end_day = int(end_date[0:4]), int(end_date[4:6]), int(end_date[6:8])
    start_date = dt_to_ms(datetime(start_year, start_month, start_day))
    end = dt_to_ms(datetime(end_year, end_month, end_day))
    # query from start_date to end_date
    dfs = []
    if interval == '1h':
        time_step = 1*60*60*1000
    elif interval == '1m':
        time_step = 1*60*1000
    elif interval == '1d':
        time_step = 1*24*60*60*1000
    end_date = start_date-time_step
    while (end_date < end):
        start_date = end_date + time_step
        end_date = start_date + (limit-500)*time_step
        if end_date >= end:
            end_date = end
        suff = '/dapi/v1/klines' if isCoin else '/fapi/v1/klines'
        suff = suff if not isSpot else '/api/v3/klines'
        suff += '?symbol='+pair+'&startTime='+str(start_date)+'&endTime='+str(end_date)+'&limit='+str(limit)+'&interval='+interval
        url = COIN_API if isCoin else USD_API
        url = url if not isSpot else SPOT_API
        result = re.get(url+suff, headers=headers)
        data = result.json()
        closePrice, time = [], []
        for ohlc in data:
            closePrice.append(float(ohlc[4]))
            time.append(dt_from_ms(ohlc[0]))
        df = pd.DataFrame({'Time':time,pair:closePrice})
        df['Time'] = pd.to_datetime(df['Time'])
        df.set_index('Time',inplace=True)
        dfs.append(df)
    df_i = dfs[0]
    for df in dfs[1:]:
        df_i = df_i.append(df)
    return df_i

In [13]:
coin_futures = get_pairs(True)
usdt_futures = get_pairs(False)
spot = get_pairs(False, True)

In [21]:
start_date, end_date = '20210101','20210407'
coin_list = spot['spot'][:5]
df = get_kline_data(False, coin_list[0], start_date, end_date, limit=1500, interval='1h', isSpot=True)
for coin in coin_list[1:]:
    print(coin)
    df_i = get_kline_data(False, coin, start_date, end_date, limit=1500, interval='1h', isSpot=True)
    if df_i.empty:
        continue
    df = df.join(df_i,how='outer')
df

ETHUSDT
BNBUSDT
BCCUSDT
NEOUSDT


,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT
Time,,,,
2021-01-01 00:00:00,28995.13,734.07,37.3764,14.451
2021-01-01 01:00:00,29409.99,748.28,37.6134,14.583
2021-01-01 02:00:00,29194.65,744.06,37.9600,14.591
2021-01-01 03:00:00,29278.40,744.82,37.9250,14.592
2021-01-01 04:00:00,29220.31,742.29,37.8702,14.636
...,...,...,...,...
2021-04-06 20:00:00,58211.84,2121.07,404.3000,61.186
2021-04-06 21:00:00,57860.63,2112.80,404.1953,61.369
2021-04-06 22:00:00,57887.24,2112.24,407.9195,62.128
